<a href="https://colab.research.google.com/github/mikexcohen/Calculus_book/blob/main/ch08_derivativesTheorems_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Calculus unraveled: Intuition, Proofs, and Python**
### Mike X Cohen (sincxpress.com)
#### https://github.com/mikexcohen/calculus_book
#### Code for Chapter 8 (Differentiation theorems)

---

# About this code file:

### This notebook contains full code solutions to the exercises in this book chapter. There are many correct ways to solve the exercises; this notebook provides *a* solution, not *THE* solution. Please use this code as a starting point to continue exploring and experimenting with calculus concepts and visualizations.

## **Using the code without the book may lead to confusion or errors.**

#### This code was written in google-colab. The notebook may require some modifications if you use a different IDE.

In [ ]:
# import libraries and define global settings
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
from IPython.display import Math

# define global figure properties used for publication
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg') # display figures in vector format
plt.rcParams.update({'font.size':14,             # font size
                     'savefig.dpi':300,          # output resolution
                     'axes.titlelocation':'left',# title location
                     'axes.spines.right':False,  # remove axis bounding box
                     'axes.spines.top':False,    # remove axis bounding box
                     'lines.linewidth':2         # increase default line thickness
                     })

# Exercise 1: An algorithm to solve the MVT

In [ ]:
def solveMVT(f,a,c):

  # Step 1: compute the derivative
  df = sym.diff(f)

  # Step 2: plug in a and c
  f_a = f.subs(x,a)
  f_c = f.subs(x,c)

  # Step 3: compute the right-hand-side of the equation for f'(b)
  RHS = (f_c-f_a) / (c-a)

  # Step 4: solve for b
  b = sym.solve(df-RHS)
  b = np.array(b).astype(np.float64)

  # Step 5: return elements of b that are between a and c
  return b[np.bitwise_and(a<b,b<c)]

In [ ]:
# test using the function from the MVT example
x = sym.symbols('x')

expr = 2*x**2 - 3*x + 1
a = -1
c = 2

solveMVT(expr,a,c)

# Exercise 2: Create an informative plot

In [ ]:
def MVTandPlot(expr,a,c):

  # solve the mean-value-theorem problem
  b = solveMVT(expr,a,c)


  # convert to callable function
  fx_lam = sym.lambdify(x,expr)
  fx_lam_df = sym.lambdify(x,sym.diff(expr))

  # get the function values
  xx = np.linspace(a-1,c+1,444)
  yy = fx_lam(xx)


  # set up the figure
  plt.figure(figsize=(7,3.5))

  # plot the function
  plt.plot(xx,yy,'k',label='f(x)')

  # plot dots for a & b
  plt.plot(a,fx_lam(a),'ko',markerfacecolor='w',markersize=9,zorder=10)
  plt.plot(c,fx_lam(c),'ko',markerfacecolor='w',markersize=9,zorder=10)

  # plot secant line
  plt.plot([a,c],[fx_lam(a),fx_lam(c)],':',color=[.3,.3,.3],label='secant')

  # plot tangent line(s)
  for i,bb in enumerate(b):
    plt.plot(bb,fx_lam(bb),'ks',markersize=9)
    tangX = [bb-1,bb+1]
    tangY = fx_lam_df(bb)*(np.array(tangX)-bb) + fx_lam(bb)
    plt.plot(tangX,tangY,'k--',label=f'tangent$_{i}$')

  # make the plot look a bit nicer
  plt.gca().set(xlim=xx[[0,-1]],xlabel='x',ylabel=r'$y=f(x)$')
  plt.legend(bbox_to_anchor=(1,1))
  plt.title(f'$f(x) = {sym.latex(expr)}$',loc='center')
  plt.show()

In [ ]:
MVTandPlot(expr,a,c)

# Exercise 3: Explore different functions

In [ ]:
# now using some different functions
expr = 2*x**3 - 3*x**2 + 1
a,c = -1,2

MVTandPlot(expr,a,c)

In [ ]:
# a sine...
expr = sym.sin(x)
a,c = 0,1.8*np.pi

MVTandPlot(expr,a,c)

In [ ]:
f = 2*x / (2*x**2+1)**2
a,c = -2,1

# quick plot
sym.plot(f)

# doesn't work...
MVTandPlot(f,a,c)

In [ ]:
# why doesn't it work? -> let's see what we're trying to solve
f_a = f.subs(x,a)
f_c = f.subs(x,c)
RHS = (f_c-f_a) / (c-a)

# solve for b
sols = sym.solve(sym.diff(f)-RHS)

# let's see if any are real-valued
for s in sols:
  print(sym.N(s))

In [ ]:
# The reason the python function crashes is that it is not designed to handle
# complex-valued solutions. You can see that this expression has two real-valued
# solutions, on either side of x=0, which confirms what sym.plot shows.
#
# If you like, you can adjust the code to handle this exception and return only
# real-valued solutions. I presented code for this in Chapter 6, although didn't
# implement it for this exercise.

# Exercise 4

In [ ]:
# "simple" trig function
f = sym.cos(x**2)
a,c = -np.pi/2,6*np.pi/7

# quick plot
sym.plot(f,(x,a,c))

# doesn't work...
MVTandPlot(f,a,c)

# # why not? -> let's see what we're trying to solve
# f_a = f.subs(x,a)
# f_c = f.subs(x,c)
# RHS = (f_c-f_a) / (c-a)

# # the expression to solve for
# sym.diff(f)-RHS

In [ ]:
sym.solve(sym.diff(f)-RHS)

# Exercise 5: Approximate a solution in numpy

In [ ]:
# lambdify the function
mvt_fun = sym.lambdify(x,sym.diff(f)-RHS) # variable 'f' is from previous exercise

# high-res approximation
xx = np.linspace(a,c,10001)
gridSearch = mvt_fun(xx)


# find critical points
from scipy.signal import find_peaks
critMinima = find_peaks(-np.abs(gridSearch))[0]
b = np.array([])
for i in critMinima:
  if np.sign(gridSearch[i-1]) + np.sign(gridSearch[i+1]) == 0:
    b = np.append(b,xx[i])



# visualize the approximation
plt.figure(figsize=(9,4))
plt.plot(xx[[0,-1]],[0,0],'--',label='Solutions touch this line',color='gray')
plt.plot(xx,gridSearch,':',color=[.7,.7,.7],label="MVT objective")
plt.plot(xx,np.abs(gridSearch),'k',label="|MVT objective|")
plt.plot(b,mvt_fun(b),'ko',markersize=9,markerfacecolor='w',label='Solutions')

plt.legend()
plt.gca().set(xlim=xx[[0,-1]],xlabel='x',ylabel='y')

plt.tight_layout()
plt.savefig('diffTheorems_ex5a.png')
plt.show()

In [ ]:
# convert to function
funLambda = sym.lambdify(x,f)
funLambda_df = sym.lambdify(x,sym.diff(f))

# get the function values
xx = np.linspace(a-2,c+2,400)
yy = funLambda(xx)


# set up the figure
plt.figure(figsize=(12,5))

# plot the function
plt.plot(xx,yy,'k',label='f(x)')

# plot dots for a & c
plt.plot([a,c],funLambda(np.array([a,c])),'ko',markerfacecolor='w',markersize=9)

# plot secant line
plt.plot([a,c],[funLambda(a),funLambda(c)],':',color=[.7,.7,.7],label='Secant',zorder=-4)

# plot the tangent lines
for i,bb in enumerate(b):

  # plot the points
  plt.plot(bb,funLambda(bb),'ks',markerfacecolor='gray',markersize=8)

  # create and plot a tangent line with slope = f'
  tangX = [bb-1,bb+1]
  tangY = funLambda_df(bb)*(np.array(tangX)-bb) + funLambda(bb)
  plt.plot(tangX,tangY,'k--',label=f'tangent$_{i}$',zorder=-10)

# make the plot look a bit nicer
plt.gca().set(xlim=xx[[0,-1]],xlabel='x',ylim=[-1.2,1.2],ylabel=r'$y=f(x)$')
plt.legend(bbox_to_anchor=(1,1))
plt.title(f'$f(x) = {sym.latex(f)}$',loc='center')

plt.tight_layout()
plt.savefig('diffTheorems_ex5b.png')
plt.show()

# Exercise 8.6: Rolle's theorem

In [ ]:
# the function
x = sym.symbols('x',real=True)
fx = sym.Abs(sym.sin(x))

# the bounds
a = np.pi/2
c = 3*np.pi/2

# step 1: confirm that f(a)=f(c)
display(Math('f(x) = %s' %sym.latex(fx))), print('')
display(Math('f\\left(%s\\right) = %s' %(sym.latex(a),sym.latex(fx.subs(x,a))))), print('')
display(Math('f\\left(%s\\right) = %s' %(sym.latex(c),sym.latex(fx.subs(x,c)))))

In [ ]:
# step 2: find the derivative
df = sym.diff(fx)
display(Math("f'(x) = %s" %sym.latex(df))), print('')

# and solve for f'=0
b = sym.solve(df)[0]
display(Math("f'(%s) = %s" %(b,df.subs(x,b))))

In [ ]:
# let's see the function!
sym.plot(fx,(x,a,c));

In [ ]:
# Based on the plot, finding the minimum is easy:

# lambdify the function
fx_lam = sym.lambdify(x,fx)

# grid search
xx = np.linspace(a,c,2531)
b = xx[ np.argmin(fx_lam(xx)) ]

print(b) # it's pi :)

In [ ]:
# lambdify the function
fx_lam = sym.lambdify(x,fx)

# plot
xx = np.linspace(0,2*np.pi,253)

plt.figure(figsize=(9,3))

plt.plot(xx,fx_lam(xx),'k',label=r'$f(x)=%s$'%sym.latex(fx))
plt.plot(a,fx_lam(a),'ko',markerfacecolor='gray',markersize=11,label='a')
plt.plot(c,fx_lam(c),'ks',markerfacecolor='gray',markersize=11,label='c')
plt.plot(b,fx_lam(b),'kv',markerfacecolor='gray',markersize=11,label='b')


plt.gca().set(xlim=xx[[0,-1]],xlabel='x',ylabel='y = f(x)')
plt.legend()

plt.tight_layout()
plt.savefig('diffTheorems_ex6.png')
plt.show()

# Exercise 8.7

In [ ]:
x = np.linspace(0,10*np.pi,1001)
yc = x + 0
ys = x + 0

_,axs = plt.subplots(1,3,figsize=(12,4))

for i in range(10):

  yc = np.cos(yc)
  ys = np.sin(ys)

  axs[0].plot(x,yc,color=np.full(3,10/12-i/12))
  axs[1].plot(x,ys,color=np.full(3,10/12-i/12))
  axs[2].plot(yc,ys,color=np.full(3,10/12-i/12))


axs[0].set(title=r'$\bf{A}$)  Iterative cosines',xlabel='$\\theta$',ylabel='y',xlim=x[[0,-1]])
axs[1].set(title=r'$\bf{B}$)  Iterative sines',xlabel='$\\theta$',ylabel='y',xlim=x[[0,-1]])
axs[2].set(title=r'$\bf{C}$)  cos by sin',xlabel='Cosines',ylabel='Sines')

plt.tight_layout()
plt.savefig('diffTheorems_ex7.png')
plt.show()